# BALTO Graphical User Interface (prototype)

This Jupyter notebook creates a GUI (graphical user interface) for the BALTO (Brokered Alignment of Long-Tail Observations) project.  BALTO is funded by the NSF EarthCube program.  The GUI aims to provide a simplified and customizable method for users to access data sets of interest on servers that support the OpenDAP data access protocol.  This interactive GUI runs within the Jupyter notebook and uses the Python packages: <b>ipywidgets</b> (for widget controls), <b>ipyleaflet</b> (for interactive maps) and <b>pydap</b> (an OpenDAP client).

The Python source code to create the GUI and to process events is in a Python module called <b>balto_gui.py</b> that must be copied into the same directory as this Jupyter notebook.

This is an <b>accordion-style</b> GUI, which allows you to switch between GUI panels without scrolling in the notebook.

You can run this notebook in a browser window without installing anything on your computer, using something called <b>Binder</b>.  On GitHub, you will see a Binder icon and a link labeled "Launch Binder".

To run this Jupyter notebook without Binder, it is recommended to install Python 3.7 from an Anaconda distribution and to then create a <b>conda environment</b> called <b>balto</b>.  Instructions for how to create a conda environment are given in Appendix 1.

#### Table of Contents

1.  <a href="#start_gui">Import and Start the BALTO GUI</a> <br>
2.  <a href="#get_data">Download Some Data</a> <br>
3.  <a href="#explore">Explore and Plot Data</a> <br>
4.  <a href="#var_match">Matching Variable Names to Standardized Names</a> <br>
5.  <a href="#low_level">Low-level Access to GUI Settings</a> <br>
6.  <a href="#references">References for More Info</a> <br>
7.  <a href="#setup">Appendix 1: Set up a conda environment called "balto"</a> <br>

## Import and Start the BALTO GUI  <a id="start_gui"></a>

In [1]:
import balto_gui as bg
balto = bg.balto_gui()
balto.show_gui()

Output()

## Download Some Data  <a id="get_data"></a>

## Explore and Plot Data  <a id="explore"></a>

## Matching Variable Names to Standardized Names  <a id="var_match"></a>

In [2]:
# This uses an SVO service at:  http://34.73.227.230:8000/match_phrase/
balto.get_possible_svo_names('air_temp', SHOW_IRI=False)


label = air__temperature
rank  = 0.5

label = air__relative_permittivity
rank  = 0.458

label = soil_air__volume_fraction
rank  = 0.438

label = atmosphere_air__temperature
rank  = 0.438

label = air__yearly_average_of_temperature
rank  = 0.438

label = air__volume_kinematic_viscosity
rank  = 0.438

label = air__volume_dynamic_viscosity
rank  = 0.438

label = air__shear_kinematic_viscosity
rank  = 0.438

label = air__shear_dynamic_viscosity
rank  = 0.438

label = air__daily_min_of_temperature
rank  = 0.438

label = air__daily_max_of_temperature
rank  = 0.438

label = snowpack_top_air__temperature
rank  = 0.425

label = sea_surface_air__temperature
rank  = 0.425

label = sea_surface_air__pressure
rank  = 0.425

label = land_surface_air__temperature
rank  = 0.425



## Low-Level Access to the GUI Settings   <a id="low_level"></a>

In [3]:
print(balto.map_minlon_box.value)

-180.0


## References for More Info  <a id="references"></a>

A Good Anaconda Reference

A Good BALTO Reference <br>
https://www.earthcube.org/group/brokered-alignment-long-tail-observations-balto <br>
https://www.nsf.gov/awardsearch/showAward?AWD_ID=1740704

A Good Binder Reference <br>
https://github.com/binder-examples

A Jupyter Notebook Reference

A Good OpenDAP Reference <br>
https://www.opendap.org/ <br>
https://opendap.github.io/documentation/QuickStart.html <br>
https://www.opendap.org/support/user-documentation

<b>NetCDF CF Conventions</b> <br>
http://cfconventions.org/cf-conventions/cf-conventions.html

<b>appmode</b> Jupyter plugin (not used) <br>
https://github.com/oschuett/appmode <br>
https://github.com/binder-examples/appmode

<b>traitlets</b> (used by ipywidgets) <br>
https://traitlets.readthedocs.io/en/stable/

The <b>ipywidgets</b> Python package <br>
https://ipywidgets.readthedocs.io/en/latest/index.html <br>
https://ipywidgets.readthedocs.io/en/latest/user_guide.html

The <b>ipyleaflet</b> Python package <br> 
https://ipyleaflet.readthedocs.io/en/latest/

The <b>pydap</b> Python package <br>
https://www.pydap.org/en/latest/ <br>
https://github.com/pydap/pydap

<b>Jupyter Lab</b> <br>
https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html <br>
https://jupyterlab.readthedocs.io/en/stable/user/extensions.html <br>
https://ipywidgets.readthedocs.io/en/latest/user_install.html#installing-the-jupyterlab-extension<br>

<b>Scientific Variables Ontology</b> (SVO) <br>
http://www.geoscienceontology.org/ <br>
Cite Stoica and Peckham papers.


## Appendix 1:  Set up a conda environment called "balto"    <a id="setup"></a>

To run this Jupyter notebook, it is recommended to use Python 3.7 from an Anaconda distribution and to create a "balto" conda environment with the following commands.  This helps to prevent conflicts with other Python packages you may have installed.  After installing Anaconda, use these commands to create the environment. 

``` bash
% conda update -n base -c defaults conda
% conda create --name balto
% conda activate balto
% conda install -c conda-forge ipywidgets
% conda install -c conda-forge ipyleaflet
% conda install -c conda-forge pydap
```

If you want to use this notebook in Jupyter Lab (vs. Jupyter Notebook), you need to install a few more Python packages and extensions, as follows.

``` bash
% conda install -c conda-forge jupyterlab
% conda install -c conda-forge nb_conda_kernels  # (needed for conda envs)

% conda install -c conda-forge nodejs
% conda install -c conda-forge widgetsnbextension
% jupyter labextension install jupyter-leaflet
% jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

When you want to run the notebook, type "conda activate balto" to activate this environment.  Then change to the directory that contains BALTO_GUI.ipynb and type "jupyter notebook" or "jupyter lab".

Finally, choose this notebook by name, "BALTO_GUI.ipynb", and make sure to choose the kernel called:  Python [conda env:balto].  See the References section at the end for more info.